In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://apdi.autocaredata.com/items/' + input_.loc[a, 'Part_Number'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                if len(html.xpath('//div[@class="carousel-item active"]/div[@class="image"]/img/@src')) == 0:
                    src = ''
                else:
                    src = html.xpath('//div[@class="carousel-item active"]/div[@class="image"]/img/@src')[0].strip()

                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@id="collapseBuyersGuideDetail"]/table/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for tr in list_tr:
                    make = tr.xpath('./td[1]/text()')[0].strip()
                    model = tr.xpath('./td[2]/text()')[0].strip()
                    engine = tr.xpath('./td[3]/ul/li[2]/text()')[0].strip()
                    year = tr.xpath('./td[4]/text()')[0].strip()

                    list_vehicle.append(make + ' ' + model + ' ' + year + ' ' + engine)

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@id="collapseInterchanges"]/table/tbody/tr')
                
                # = = = = = = = = = = = = = = =
                
                dict_oe = {}
                for tr in list_tr:
                    key = tr.xpath('./td[2]/text()')[0].strip()

                    if key not in dict_oe:
                        dict_oe[key] = [tr.xpath('./td[1]/text()')[0].strip()]
                    else:
                        dict_oe[key].append(tr.xpath('./td[1]/text()')[0].strip())

                    dict_oe[key] = sorted(list(set(dict_oe[key])))
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@class="item_attributes"]/table/tr/td/table/tr')

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                         'Vehicle': vehicle,
                                         'Src': src}])

                for key in dict_oe:
                    df_temp.loc[0, 'OE_' + key] = ';'.join(dict_oe[key])

                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./td/text()')[0].strip()[:-1]] = tr.xpath('./th/text()')[0].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Part_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./info-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./info_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2

[ok] - 3310714
[尝试次数：1] - [剩余数量：0] - [当前时间：17:57:06]

[ok] - 3310721
[尝试次数：1] - [剩余数量：0] - [当前时间：17:57:07]

输出ing...

Done ~
